In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'tipodepropiedad': str, 'idzone':float}


training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','tipodepropiedad','idzona'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv', dtype = types, usecols=['id','tipodepropiedad','idzona'])

training_data.fillna(value = {'tipodepropiedad':'sin_tipo', 'idzona':'sin_zona'}, inplace = True)
evaluation_data.fillna(value = {'tipodepropiedad':'sin_tipo', 'idzona':'sin_zona'}, inplace = True)

In [2]:
def tipo_de_prop(tipo):
    if tipo in ['Terreno comercial', 'Local comercial', 'Oficina comercial', 'Local en centro comercial', 'Bodega comercial', 'Inmuebles productivos urbanos']: return 'comercial'
    if tipo in ['Apartamento', 'Casa en condominio', 'Casa', 'Terreno', 'Edificio', 'Casa uso de suelo', 'Duplex', 'Departamento Compartido']: return 'residencial'
    if tipo in ['Terreno industrial', 'Nave industrial']: return 'industrial'
    if tipo in ['Huerta', 'Rancho']: return 'rural'
    else: return 'sin_tipo'

In [3]:
training_data['tipo'] = training_data.tipodepropiedad.apply(lambda x: tipo_de_prop(x))
evaluation_data['tipo'] = evaluation_data.tipodepropiedad.apply(lambda x: tipo_de_prop(x))

In [4]:
training_data.head()

,id,tipodepropiedad,idzona,tipo
0,254099,Apartamento,23533,residencial
1,53461,Casa en condominio,24514,residencial
2,247984,Casa,48551,residencial
3,209067,Casa,53666,residencial
4,185997,Apartamento,47835,residencial


In [5]:
tipo_de_zona_evaluation = evaluation_data.groupby('idzona')['tipo'].value_counts(True).to_frame().unstack()
tipo_de_zona_evaluation.columns = tipo_de_zona_evaluation.columns.get_level_values(1).astype(str)
tipo_de_zona_evaluation['tipo_de_zona'] = tipo_de_zona_evaluation.T.idxmax()
tipo_de_zona_evaluation = tipo_de_zona_evaluation.reset_index()

In [6]:
tipo_de_zona_evaluation = tipo_de_zona_evaluation[['idzona','tipo_de_zona']]
tipo_de_zona_evaluation

tipo,idzona,tipo_de_zona
0,22,residencial
1,23,residencial
2,26,residencial
3,30,residencial
4,32,residencial
...,...,...
11229,5.0004e+07,residencial
11230,5.0004e+07,residencial
11231,5.0004e+07,residencial
11232,5.0004e+07,residencial


In [7]:
tipo_de_zona_training = training_data.groupby('idzona')['tipo'].value_counts(True).to_frame().unstack()
tipo_de_zona_training.columns = tipo_de_zona_training.columns.get_level_values(1).astype(str)
tipo_de_zona_training['tipo_de_zona'] = tipo_de_zona_training.T.idxmax()
tipo_de_zona_training = tipo_de_zona_training.reset_index()

In [8]:
tipo_de_zona_training = tipo_de_zona_training[['idzona','tipo_de_zona']]
tipo_de_zona_training

tipo,idzona,tipo_de_zona
0,22,residencial
1,23,residencial
2,26,residencial
3,27,residencial
4,28,residencial
...,...,...
19715,5.0004e+07,residencial
19716,5.0004e+07,residencial
19717,5.0004e+07,residencial
19718,5.0004e+07,residencial


In [9]:
training_data = training_data.merge(tipo_de_zona_training, on='idzona', how = 'left')
training_data = training_data[['id','tipo_de_zona']]

In [10]:
training_data.tipo_de_zona.value_counts()

residencial    238444
comercial         832
sin_tipo          600
rural              97
industrial         27
Name: tipo_de_zona, dtype: int64

In [11]:
evaluation_data = evaluation_data.merge(tipo_de_zona_evaluation, on='idzona', how = 'left')
evaluation_data = evaluation_data[['id','tipo_de_zona']]

In [12]:
evaluation_data.tipo_de_zona.value_counts()

residencial    59325
comercial        429
sin_tipo         214
rural             27
industrial         5
Name: tipo_de_zona, dtype: int64

In [13]:
evaluation_data

,id,tipo_de_zona
0,4941,residencial
1,51775,residencial
2,115253,residencial
3,299321,residencial
4,173570,residencial
...,...,...
59995,75094,residencial
59996,171847,residencial
59997,138313,residencial
59998,271268,residencial


In [14]:
training_entities = training_data[['tipo_de_zona']].to_numpy()
evaluation_entities = evaluation_data[['tipo_de_zona']].to_numpy()
all_entities = np.unique( np.concatenate( (training_entities, evaluation_entities), axis=None) )

In [15]:
for entity in all_entities:
    training_data['entity_'+entity] = training_data['tipo_de_zona'].apply(lambda x: 1 if x == entity else 0)
    evaluation_data['entity_'+entity] = evaluation_data['tipo_de_zona'].apply(lambda x: 1 if x == entity else 0)
    
training_data.drop(columns=['tipo_de_zona'], inplace = True)
evaluation_data.drop(columns=['tipo_de_zona'], inplace = True)

training_data.head()

,id,entity_comercial,entity_industrial,entity_residencial,entity_rural,entity_sin_tipo
0,254099,0,0,1,0,0
1,53461,0,0,1,0,0
2,247984,0,0,1,0,0
3,209067,0,0,1,0,0
4,185997,0,0,1,0,0


In [17]:
training_data.to_csv('../../res/ftr/encoded_type_of_idzone_train.csv')
evaluation_data.to_csv('../../res/ftr/encoded_type_of_idzone_evaluation.csv')